In [3]:
# imports
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torchvision.models as models
import numpy as np
from sklearn import decomposition

from art.attacks.evasion import FastGradientMethod, CarliniL2Method, ProjectedGradientDescentPyTorch, DeepFool
from art.estimators.classification import PyTorchClassifier

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(0)
np.random.seed(0)

# Model
model = models.resnet18(pretrained=True).eval()
preprocessing = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)

## Data
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

imagenet_dataset = datasets.ImageFolder(
    "/Users/aparnagupte/Downloads/imagenet10-val",
     transforms.Compose([
            transforms.CenterCrop(224),
            # transforms.RandomResizedCrop(224),
            # transforms.RandomHorizontalFlip(),
            ## I removed the randomness
            transforms.ToTensor(),
            normalize,
        ]))

loader = DataLoader(imagenet_dataset, batch_size=1, shuffle=False)
label_map = [2, 31, 34, 61, 99, 121, 208, 281, 309, 388]

for images, labels in loader:
    print('shape of images: ', images.shape)
    labels = np.array([label_map[i] for i in labels])
    min_pixel_value, max_pixel_value = torch.amin(images), torch.amax(images)
    images = images.detach().cpu().numpy()
    break

print(labels)

# Step 2a: Define the loss function and the optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Step 3: Create the ART classifier

classifier = PyTorchClassifier(
    model=model,
    clip_values=(min_pixel_value, max_pixel_value),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(1, 224, 224),
    nb_classes=1000, # imagenet10, this is true
)

# Step 5: Evaluate the ART classifier on benign test examples

predictions = classifier.predict(images)
predictions = np.argmax(predictions, axis=1)
print('shape of predictions: ', predictions.shape)

# Step 6: Generate adversarial test examples
attack = FastGradientMethod(estimator=classifier, eps=0.1, targeted=True)
# attack = CarliniL2Method(classifier=classifier)
# attack = ProjectedGradientDescentPyTorch(estimator=classifier, eps=0.05, eps_step = 0.01/12, max_iter=50)
# attack = DeepFool(classifier=classifier, max_iter=80, epsilon=0.01)

adversarial_images = attack.generate(x=images, y=np.array([494, 494]))

# print('attack doing anything?? ', images == adversarial_images) # seems to be doing something, the right thing even!

print('shape of adversarial_images: ', adversarial_images.shape)

# Step 7: Evaluate the ART classifier on adversarial test examples

attacked_predictions = classifier.predict(adversarial_images)
attacked_predictions = np.argmax(attacked_predictions, axis=1)
print('attacked_predictions: ', attacked_predictions)

success_rate = np.sum(predictions != attacked_predictions) / len(labels)
print("Success rate: {}%".format(success_rate * 100))

## Function to compute (k,p) points
pca = decomposition.PCA()

def plot_images(X):
    # X.shape = [rows, columns, channels]
    channels, rows, columns = X.shape
    X = np.swapaxes(X, 0, 1)
    X = np.swapaxes(X, 1, 2)
    # X = X.detach().cpu().numpy()
    plt.imshow(X) # takes input of shape [rows, columns, channels]
    plt.show()

plot_images(images[1])
plot_images(adversarial_images[1])

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/aparnagupte/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

shape of images:  torch.Size([1, 3, 224, 224])
[2]


ModuleNotFoundError: No module named 'tensorflow'